In [ ]:
!pip install -q --upgrade pip
!pip install -U jaxlib==0.4.4+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 MB 5.5 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.2+cuda11.cudnn82
    Uninstalling jaxlib-0.4.2+cuda11.cudnn82:
      Successfully uninstalled jaxlib-0.4.2+cuda11.cudnn82


  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [2]:
from network_config import get_network_configuration
from train import cross_validate, cross_entropy, save_GD_raw_results, save_core_results

from jax import numpy as jnp
import matplotlib.pyplot as plt
from jax import jit, grad, vmap

import numpy as np
import os
import pandas as pd

In [4]:
# MUTAG 2 WL

utc_time = pd.Timestamp.utcnow()
dataset_name = "MUTAG"
training_method = "Gradient_Descent"
nn_type = "twl"

preprocessed_base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"
repo_path = "/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK"

cv_folds = 10
learning_rate =  0.0005
epochs = 200

# load preprocessed data
Y = jnp.load(preprocessed_base_path + f"/ys.npy")
graphs_edge_features = jnp.load(preprocessed_base_path + f"/graphs_edge_features.npy")
graph_two_wl_pattern = jnp.load(preprocessed_base_path + f"/two_wl_pattern_radius_1.npy")

# load the network configuration
init_fn, apply_fn, kernel_fn = get_network_configuration(dataset_name, nn_type, training_method)

# loss function
loss = jit(cross_entropy)
grad_loss = jit(grad(lambda params, x, y, pattern: loss(y, apply_fn(params, x, pattern=pattern))))

# cross validation training, to report validation acc mean and std
train_results = cross_validate(
    graphs_edge_features,
    Y,
    graph_two_wl_pattern,
    cv_folds,
    init_fn,
    apply_fn,
    learning_rate,
    epochs,
    nn_type,
    loss,
    grad_loss)


save_GD_raw_results(
    dataset_name, training_method, nn_type, epochs, utc_time.timestamp(), repo_path, cv_folds, train_results
)
save_core_results(np.array([np.array(i) for i in train_results[3]]), utc_time, dataset_name, training_method, nn_type, cv_folds, repo_path)

Start CV fold: 0
nb train samples: 171 | nb val samples: 19
	 train loss: 110.92 | val loss: 134.46 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 80.77 | val loss: 98.03 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 50.78 | val loss: 61.79 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 16.91 | val loss: 20.79 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 5.91 | val loss: 3.49 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 5.39 | val loss: 3.18 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 1.96 | val loss: 2.57 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 1.40 | val loss: 0.82 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 0.98 | val loss: 1.25 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 0.66 | val loss: 0.56 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 0.68 | val loss: 0.53 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 0.63 | val loss: 0.60 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 0.63 | val loss: 0.62 | trai

In [5]:
# MUTAG GCN

utc_time = pd.Timestamp.utcnow()
dataset_name = "MUTAG"
training_method = "Gradient_Descent"
nn_type = "gcn"

preprocessed_base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"
repo_path = "/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK"

cv_folds = 10
learning_rate =  0.0001
epochs = 200

# load preprocessed data
Y = jnp.load(preprocessed_base_path + f"/ys.npy")
graps_node_features = np.load(preprocessed_base_path + f"/graps_node_features.npy")
graph_conv_pattern = np.load(preprocessed_base_path + f"/graph_conv_pattern.npy")
graps_node_features = np.swapaxes(graps_node_features, 2,3)

# load the network configuration
init_fn, apply_fn, kernel_fn = get_network_configuration(dataset_name, nn_type, training_method)

# loss function
loss = jit(cross_entropy)
grad_loss = jit(grad(lambda params, x, y, pattern: loss(y, apply_fn(params, x, pattern=pattern))))

# cross validation training, to report validation acc mean and std
train_results = cross_validate(
    graps_node_features,
    Y,
    graph_conv_pattern,
    cv_folds,
    init_fn,
    apply_fn,
    learning_rate,
    epochs,
    nn_type,
    loss,
    grad_loss)


save_GD_raw_results(
    dataset_name, training_method, nn_type, epochs, utc_time.timestamp(), repo_path, cv_folds, train_results
)
save_core_results(np.array([np.array(i) for i in train_results[3]]), utc_time, dataset_name, training_method, nn_type, cv_folds, repo_path)

Start CV fold: 0
nb train samples: 171 | nb val samples: 19


/usr/local/lib/python3.8/dist-packages/neural_tangents/_src/stax/linear.py:439: UserWarning: Negative indices in `pattern` are considered as padding (i.e. ignored), unlike typical numpy negative indexing.
  warnings.warn('Negative indices in `pattern` are considered as padding '


	 train loss: 34.90 | val loss: 34.81 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 31.32 | val loss: 31.17 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 27.75 | val loss: 27.54 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 24.18 | val loss: 23.93 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 20.62 | val loss: 20.35 | train acc: 0.3509 | val acc: 0.2632
	 train loss: 17.05 | val loss: 16.78 | train acc: 0.3509 | val acc: 0.2632
	 train loss: 13.49 | val loss: 13.25 | train acc: 0.3567 | val acc: 0.2632
	 train loss: 9.98 | val loss: 9.78 | train acc: 0.3801 | val acc: 0.3684
	 train loss: 6.60 | val loss: 6.42 | train acc: 0.4211 | val acc: 0.3684
	 train loss: 3.48 | val loss: 3.26 | train acc: 0.4737 | val acc: 0.4211
	 train loss: 1.07 | val loss: 0.88 | train acc: 0.6608 | val acc: 0.6842
	 train loss: 0.67 | val loss: 0.50 | train acc: 0.7544 | val acc: 0.7895
	 train loss: 0.62 | val loss: 0.44 | train acc: 0.7836 | val acc: 0.8421
	 train loss: 0.57 | val